Noa Hen - 207422221
Romi Pitshon - 314919614

In [273]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')
stop_words = set(stopwords.words("english"))
from sklearn.model_selection import train_test_split
import numpy as np
from tabulate import tabulate

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\romip\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\romip\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\romip\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [274]:
# Load the dataset
df = pd.read_csv('spam_ham_dataset.csv')

In [275]:
df['label'].value_counts()

ham     3672
spam    1499
Name: label, dtype: int64

In [276]:
#Creating variables to perform tokenization
lemmatizer = WordNetLemmatizer()
tokenizer = nltk.RegexpTokenizer(r"\w+")

In [277]:
#A function that receives the email messages from the file and turns it into a list of tokens
def message_to_token_list(s):
  tokens = tokenizer.tokenize(s)
  lowercased_tokens = [t.lower() for t in tokens]
  lemmatized_tokens = [lemmatizer.lemmatize(t) for t in lowercased_tokens]
  useful_tokens = [t for t in lemmatized_tokens if t not in stop_words]

  return useful_tokens

In [278]:
#split the data to train set and test set
df = df.sample(frac=1, random_state=1)
df = df.reset_index(drop=True)

split_index = int(len(df) * 0.8)
train_df, test_df = df[:split_index], df[split_index:]

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df, test_df

(      Unnamed: 0 label                                               text  \
 0           3430   ham  Subject: what the heck\r\ndaren ,\r\nnow what ...   
 1           2070   ham  Subject: hilcorp old ocean volume\r\naccording...   
 2           3974  spam  Subject: jurirne get latest softwares , 99 % s...   
 3           1502   ham  Subject: sitara patch\r\na patch is being rele...   
 4           4591  spam  Subject: archived great shots of california li...   
 ...          ...   ...                                                ...   
 4131        4083  spam  Subject: viewsonic airpanel vl 50 15 - inch sm...   
 4132        1297   ham  Subject: fw : why you shouldn ' t piss on the ...   
 4133        2057   ham  Subject: holiday invitation\r\nplease click on...   
 4134        2398   ham  Subject: managing director and vice president ...   
 4135         805   ham  Subject: eastrans nom - 5 / 24 / 2000\r\nthis ...   
 
       label_num  
 0             0  
 1             0  
 2   

In [279]:
#Performing an EDA that samples the information and shows the frequency of tokens for each category
token_counter = {}

for message in train_df['text']:
  message_as_token_lst = message_to_token_list(message)

  for token in message_as_token_lst:
    if token in token_counter:
      token_counter[token] += 1
    else:
      token_counter[token] = 1

len(token_counter)

42460

In [280]:
token_counter

{'subject': 6425,
 'heck': 2,
 'daren': 1460,
 'see': 973,
 'ken': 101,
 'back': 488,
 'jeff': 102,
 'development': 206,
 'ha': 1394,
 'lot': 95,
 'bud': 4,
 'high': 305,
 'place': 244,
 'could': 447,
 'many': 244,
 'change': 923,
 'horizon': 3,
 'moved': 73,
 'new': 1170,
 'building': 47,
 'yet': 124,
 'looking': 302,
 'tour': 24,
 'invitation': 46,
 'kid': 27,
 'wife': 54,
 'home': 272,
 'read': 174,
 'work': 445,
 'project': 191,
 'jen': 5,
 'couple': 65,
 'week': 430,
 'heading': 9,
 'waco': 6,
 'going': 238,
 'law': 47,
 'school': 64,
 'built': 30,
 'tobacco': 7,
 'settlement': 122,
 'money': 299,
 'feel': 157,
 'like': 533,
 'contributators': 1,
 'four': 72,
 'quarter': 59,
 '5': 1256,
 'maybe': 43,
 '6': 857,
 'go': 440,
 'son': 25,
 'still': 251,
 'florida': 31,
 'plan': 242,
 'run': 94,
 'september': 248,
 'day': 1239,
 'disneyworld': 1,
 'jeni': 1,
 'supposed': 14,
 'get': 1058,
 'u': 1096,
 'ticket': 663,
 'nebraska': 3,
 'game': 80,
 'year': 416,
 'fun': 59,
 'speaking': 12

In [281]:
#Selecting the features according to the threshold value
def keep_token(proccessed_token, threshold):
  if proccessed_token not in token_counter:
    return False
  else:
    return token_counter[proccessed_token] > threshold

In [282]:
features = set()

for token in token_counter:
  if keep_token(token, 3000):
    features.add(token)

features

{'2000', 'com', 'ect', 'enron', 'hou', 'subject'}

In [283]:
features = list(features)
features

['ect', 'hou', 'com', '2000', 'subject', 'enron']

In [284]:
#Giving a numerical value to each token in the list
token_to_index_mapping = {t:i for t, i in zip(features, range(len(features)))}
token_to_index_mapping

{'ect': 0, 'hou': 1, 'com': 2, '2000': 3, 'subject': 4, 'enron': 5}

In [285]:
#A function that receives an email message and counts the tokens that are in the list of features
def message_to_count_vector(message):
  count_vector = np.zeros(len(features))

  processed_list_of_tokens = message_to_token_list(message)

  for token in processed_list_of_tokens:
    if token not in features:
      continue
    index = token_to_index_mapping[token]
    count_vector[index] += 1
  
  return count_vector

In [286]:
#Division into a target column and a features column
def df_to_X_y(dff):
  y = dff['label_num'].to_numpy().astype(int)

  message_col = dff['text']
  count_vectors = []

  for message in message_col:
    count_vector = message_to_count_vector(message)
    count_vectors.append(count_vector)

  X = np.array(count_vectors).astype(int)

  return X, y

In [287]:
X_train, y_train = df_to_X_y(train_df)

X_test, y_test = df_to_X_y(test_df)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((4136, 6), (4136,), (1035, 6), (1035,))

In [288]:
#normalization
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(X_train)

X_train, X_test = scaler.transform(X_train), scaler.transform(X_test)

X_train

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.02046784, 0.0239521 , 0.        , 0.03448276, 0.05263158,
        0.        ],
       [0.        , 0.        , 0.        , 0.06896552, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.00142653, 0.        , 0.        ,
        0.0075188 ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.03448276, 0.        ,
        0.0075188 ]])

In [289]:
#Building a linear regression model from the scikit-learn library, predicting and performing an accuracy assessment for the model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

lr = LogisticRegression().fit(X_train, y_train)
print(classification_report(y_test, lr.predict(X_test)))

              precision    recall  f1-score   support

           0       0.72      1.00      0.84       747
           1       0.00      0.00      0.00       288

    accuracy                           0.72      1035
   macro avg       0.36      0.50      0.42      1035
weighted avg       0.52      0.72      0.61      1035



C:\Users\romip\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\romip\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\romip\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [290]:
#Repeating the prediction model in three additional runs with a different extraction of features in order to optimize the model
features = set()

for token in token_counter:
  if keep_token(token, 1000):
    features.add(token)

features
token_to_index_mapping = {t: i for t, i in zip(features, range(len(features)))}
token_to_index_mapping


def message_to_count_vector(message):
  count_vector = np.zeros(len(features))

  processed_list_of_tokens = message_to_token_list(message)

  for token in processed_list_of_tokens:
    if token not in features:
      continue
    index = token_to_index_mapping[token]
    count_vector[index] += 1

  return count_vector


def df_to_X_y(dff):
  y = dff['label_num'].to_numpy().astype(int)

  message_col = dff['text']
  count_vectors = []

  for message in message_col:
    count_vector = message_to_count_vector(message)
    count_vectors.append(count_vector)

  X = np.array(count_vectors).astype(int)

  return X, y


X_train, y_train = df_to_X_y(train_df)

X_test, y_test = df_to_X_y(test_df)

X_train.shape, y_train.shape, X_test.shape, y_test.shape
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(X_train)

X_train, X_test = scaler.transform(X_train), scaler.transform(X_test)

X_train
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

lr = LogisticRegression().fit(X_train, y_train)
print(classification_report(y_test, lr.predict(X_test)))

              precision    recall  f1-score   support

           0       0.83      0.98      0.90       747
           1       0.89      0.49      0.63       288

    accuracy                           0.84      1035
   macro avg       0.86      0.73      0.77      1035
weighted avg       0.85      0.84      0.82      1035



In [291]:
features = set()

for token in token_counter:
  if keep_token(token, 800):
    features.add(token)

features
token_to_index_mapping = {t: i for t, i in zip(features, range(len(features)))}
token_to_index_mapping


def message_to_count_vector(message):
  count_vector = np.zeros(len(features))

  processed_list_of_tokens = message_to_token_list(message)

  for token in processed_list_of_tokens:
    if token not in features:
      continue
    index = token_to_index_mapping[token]
    count_vector[index] += 1

  return count_vector


def df_to_X_y(dff):
  y = dff['label_num'].to_numpy().astype(int)

  message_col = dff['text']
  count_vectors = []

  for message in message_col:
    count_vector = message_to_count_vector(message)
    count_vectors.append(count_vector)

  X = np.array(count_vectors).astype(int)

  return X, y


X_train, y_train = df_to_X_y(train_df)

X_test, y_test = df_to_X_y(test_df)

X_train.shape, y_train.shape, X_test.shape, y_test.shape
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(X_train)

X_train, X_test = scaler.transform(X_train), scaler.transform(X_test)

X_train
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

lr = LogisticRegression().fit(X_train, y_train)
print(classification_report(y_test, lr.predict(X_test)))

              precision    recall  f1-score   support

           0       0.93      0.94      0.94       747
           1       0.84      0.83      0.83       288

    accuracy                           0.91      1035
   macro avg       0.89      0.88      0.88      1035
weighted avg       0.91      0.91      0.91      1035



In [292]:
features = set()

for token in token_counter:
  if keep_token(token, 50):
    features.add(token)

features
token_to_index_mapping = {t: i for t, i in zip(features, range(len(features)))}
token_to_index_mapping


def message_to_count_vector(message):
  count_vector = np.zeros(len(features))

  processed_list_of_tokens = message_to_token_list(message)

  for token in processed_list_of_tokens:
    if token not in features:
      continue
    index = token_to_index_mapping[token]
    count_vector[index] += 1

  return count_vector


def df_to_X_y(dff):
  y = dff['label_num'].to_numpy().astype(int)

  message_col = dff['text']
  count_vectors = []

  for message in message_col:
    count_vector = message_to_count_vector(message)
    count_vectors.append(count_vector)

  X = np.array(count_vectors).astype(int)

  return X, y


X_train, y_train = df_to_X_y(train_df)

X_test, y_test = df_to_X_y(test_df)

X_train.shape, y_train.shape, X_test.shape, y_test.shape
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(X_train)

X_train, X_test = scaler.transform(X_train), scaler.transform(X_test)

X_train
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

lr = LogisticRegression().fit(X_train, y_train)
print(classification_report(y_test, lr.predict(X_test)))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98       747
           1       0.95      0.93      0.94       288

    accuracy                           0.97      1035
   macro avg       0.96      0.96      0.96      1035
weighted avg       0.97      0.97      0.97      1035



In [293]:
#A table showing each of the runs we performed with the amount of different features and comparing the results we received
table = [['Features:', 'Accuracy:'], [3000, 0.72], [1000, 0.84], [800, 0.91], [50, 0.97]]
print(tabulate(table, headers="firstrow"))

  Features:    Accuracy:
-----------  -----------
       3000         0.72
       1000         0.84
        800         0.91
         50         0.97


In [294]:
#Linear Regression is a machine learning algorithm based on supervised learning. It performs a regression task. Regression models a target prediction value based on independent variables. It is mostly used for finding out the relationship between variables and forecasting. Different regression models differ based on – the kind of relationship between dependent and independent variables they are considering, and the number of independent variables getting used. As we decreased the number of features, the level of accuracy increased so that we were able to reach an accuracy level of 97% using 50 features. It can be understood that in our data, the more we reduced the number of features, the better the model was able to predict the relationship between the variables.